In [1]:
import Analysis

In [2]:
data =Analysis.IntervalConfig()

In [3]:
print(data.interval_maps)

{'interval_1m': 0, 'interval_3m': 1, 'interval_5m': 2, 'interval_15m': 3}


In [4]:
print(data.target_interval)

['1m', '3m', '5m', '15m']


In [11]:
start_price = 10
target = 1500000000
while target >= start_price:
    start_price *= 1.3
    print(int(start_price))

13
16
21
28
37
48
62
81
106
137
179
232
302
393
511
665
865
1124
1461
1900
2470
3211
4175
5428
7056
9173
11925
15502
20153
26199
34059
44277
57561
74829
97278
126462
164400
213721
277837
361188
469545
610408
793531
1031590
1341068
1743388
2266405
2946326
3830224
4979292
6473079
8415003
10939505
14221356
18487763
24034092
31244320
40617616
52802901
68643771
89236903
116007974
150810366
196053476
254869519
331330375
430729487
559948334
727932834
946312684
1230206489
1599268436


In [3]:
15*0.5

7.5